In [1]:
import urllib.request, json 
import requests
import pandas as pd
import numpy as np
from scipy.stats import linregress

In [2]:
rows = ['NTC','50.0 cp ','5000.0 cp ','500.0 cp ','10.0 cp ','1000.0 cp ',
         '50.0 ng HgDNA ','100.0 cp ','10000.0 cp ','500.0 ng HgDNA ']
ind = ['Expt','qPCR Plate','Reps']
dep = ['ID Template Name','#Ct Pos','#Ct Valid', '#Tm Specif', '#Tm PD', '#Tm NS', 'Mean Ct', 'Min Ct',
       'Max Ct', 'Min Tm1', 'Max Tm1','Specif ng/ul Pos', 'Mean Specif ng/ul','Mean NS ng/ul', 'Mean PD ng/ul'] 

In [3]:
def list_to_string(value):
    if not isinstance(value,float):
        return "".join(str(e) for e in value)
    return None

In [4]:
def dataframe_list_to_string(df):
    list_cols = ['ID Assay Conc.','ID Assay Name','ID Human Conc.','ID Human Name',
               'ID Template Conc.','ID Template Name','PA Assay Conc.','PA Assay Name',
               'PA Human Conc.','PA Human Name','PA Template Conc.','PA Template Name']
    for column in list_cols:
        if column in df.columns:
            df[column] = df[column].apply(list_to_string)
    return df

In [5]:
def find_sample(row):    
    name = ''
    if row['ID Template Conc.']:
        name += '%s cp '%row['ID Template Conc.']
    if row['ID Human Conc.']:
        name += '%s ng HgDNA '%row['ID Human Conc.']
    if 'PA Template Conc.'in row and row['PA Template Conc.']:
        name += 'PA %s cp '%row['PA Template Conc.']
    if 'PA Human Conc.' in row and row['PA Human Conc.']:
        if 'PA' not in name:
            name += 'PA %s ng HgDNA '%row['PA Human Conc.']
        else:
            name += '%s ng HgDNA '%row['PA Human Conc.']
    if name =='':
        name = 'NTC'
    return name
    

In [6]:
def get_master_summary_data(experiments,well_by_id_assay):
    master_table = []
    summary_table = []
    for record in well_by_id_assay:        
        experiment_id = record['experiment_id']
        qpcr_plate_id = record['qpcr_plate_id']
        if experiment_id in experiments:
            wells =json.dumps(record['wells'].split(","))    
            _url = "https://assay-screening.herokuapp.com/api/well-results/?expt={}&plate_id={}&wells={}".format(experiment_id,qpcr_plate_id,wells)
            response =json.loads(requests.get(_url).text)
            master_table = master_table + response['master_table']
            summary_table = summary_table + response['summary_table']
    return master_table,summary_table

In [7]:
def save_master_table(df):
    possible_cols = ['qPCR Plate','qPCR Well','LC Plate','LC Well','ID Assay Name','ID Template Name',
    'ID Template Conc.','ID Human Conc.','PA Assay Name','PA Template Name',
    'PA Template Conc.','PA Human Conc.','Ct','Ct Call','Tm1', 'Tm2', 'Tm3', 'Tm4','Tm Specif','Tm NS',
    'Tm PD']
    cols_in_db = [col for col in possible_cols if col in df.columns]
    df[cols_in_db].to_csv('master.csv')

In [8]:
def calc_eff(row):
    try:                
        expected_cols = ['50.0 cp ','5000.0 cp ','500.0 cp ','10.0 cp ','100.0 cp ','1000.0 cp ','10000.0 cp ']
        cols = list(filter(lambda x : (x,'Mean Ct') in row.index ,expected_cols))        
        idx = pd.IndexSlice
        y = np.array([float(n) for n in row.loc[idx[cols, 'Mean Ct']].values])
        x = np.log10([float(n.split()[0]) for n in cols])               
        mask = ~np.isnan(x) & ~np.isnan(y)        
        
        lin_fit = linregress(x[mask], y[mask])
        eff = (10**(-1 / lin_fit.slope) - 1) * 100
    except ValueError as ve:        
        eff = 0
        
    return eff

def calc_r2(row):
    try:        
        expected_cols = ['50.0 cp ','5000.0 cp ','500.0 cp ','10.0 cp ','100.0 cp ','1000.0 cp ','10000.0 cp ']
        cols = list(filter(lambda x : (x,'Mean Ct') in row.index ,expected_cols))
        idx = pd.IndexSlice
        row = row.sort_index()
        y = np.array([float(n) for n in row.loc[idx[cols, 'Mean Ct']].values])
        x = np.log10([float(n.split()[0]) for n in cols])
        
        mask = ~np.isnan(x) & ~np.isnan(y)
        
        slope, intercept, r_value, p_value, std_err = linregress(x[mask], y[mask])
        r2 = r_value**2
    except ValueError as ve:
        r2 = np.nan
        
    return r2

def removeErrFrames(frames):
    expt_list=[]
    new_df=frames[0].T
    new_df=new_df.drop(new_df.index[0])
    for i in range(1,len(frames)):         
        try:
            new_df=new_df.append(frames[i].T)
        except:
            expt_list.append(i)
    for index in sorted(expt_list, reverse=True):
        print(frames[index])
        del frames[index]
    return frames
    

In [9]:
def get_summary_by_sample(df,rows,ind,dep):
    idx = pd.IndexSlice
    df = df[df['qPCR Plate']!='A81_E214_1_ID'] 
    frames = []
# ['Expt','plate_id','Group','templates']
    for n, grp in df.groupby(['qPCR Plate','ID Assay Name']):    
        a = pd.DataFrame(grp.set_index('sample').loc[rows].stack())
        a = a.sort_index(level=[0, 1])
        a = a.sort_index()
        b = a.loc[idx['NTC', ind], :]
        b.index = pd.MultiIndex.from_product([['Common'], b.index.get_level_values(1)])
        c = a.loc[idx[:, dep], :]

        df = pd.concat([b, c])
        df.columns = pd.MultiIndex.from_tuples([n])
        #df = df.rename(columns={0: n[2]})
        frames.append(df)
    return frames

In [10]:
def calculate_eff_r2(df,rows):
    frames=removeErrFrames(df)
    frames_df = pd.concat(frames, axis=1).reindex(rows, level=0)
    frames_df = frames_df.sort_index().T
    frames_df[('Common', 'Efficiency')] = frames_df.apply(calc_eff, axis=1)
    frames_df[('Common', 'R Squared')] = frames_df.apply(calc_r2,axis=1)
    return frames_df

In [11]:
def get_summary(experiments,rows,ind,dep):
    with urllib.request.urlopen("https://assay-screening.herokuapp.com/api/well-aggregate/") as url:
        well_by_id_assay = json.loads(url.read().decode())
        master_table,summary_table = get_master_summary_data(experiments,well_by_id_assay)
        master_df = dataframe_list_to_string(pd.DataFrame(master_table))
        summary_df = dataframe_list_to_string(pd.DataFrame(summary_table))
        summary_df['sample'] = summary_df.apply(find_sample,axis=1)
        save_master_table(master_df)
        summary_by_sample=get_summary_by_sample(summary_df,rows,ind,dep)
        summary_with_eff_r2 = calculate_eff_r2(summary_by_sample,rows)
        summary_with_eff_r2 = summary_with_eff_r2.sort_index(axis=1, level=[0, 1])
        summary_with_eff_r2.to_csv('group_by_columns.csv')
        return summary_with_eff_r2

In [12]:
experiments = ['A81_E303']
get_summary(experiments,rows,ind,dep)

c:\users\prasanna.gnanaraj\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  import sys


10.0 cp                           \
                                     #Ct Pos #Ct Valid #Tm NS #Tm PD   
A81_E303_1 AMR_NDM_10.x_NDM31_NDM39    4 | 4     4 | 4  0 | 4  0 | 4   
           AMR_NDM_12.x_NDM32_NDM39    4 | 4     4 | 4  0 | 4  0 | 4   
           AMR_NDM_14.x_NDM40_NDM48    3 | 4     4 | 4  0 | 4  0 | 4   
           AMR_NDM_16.x_NDM41_NDM47    0 | 4     4 | 4  0 | 4  1 | 4   
           AMR_NDM_18.x_NDM41_NDM49    0 | 4     4 | 4  0 | 4  0 | 4   
           AMR_NDM_7.x_NDM25_NDM30     0 | 4     4 | 4  0 | 4  0 | 4   
A81_E303_2 AMR_NDM_x.10_NDM42_NDM46    0 | 4     4 | 4  0 | 4  1 | 4   
           AMR_NDM_x.5_NDM27_NDM29     0 | 4     4 | 4  2 | 4  0 | 4   
           AMR_NDM_x.6_NDM33_NDM35     0 | 4     4 | 4  0 | 4  4 | 4   
           AMR_NDM_x.7_NDM33_NDM36     0 | 4     4 | 4  0 | 4  2 | 4   
           AMR_NDM_x.8_NDM33_NDM37     0 | 4     4 | 4  1 | 4  4 | 4   
           AMR_NDM_x.9_NDM42_NDM45     3 | 4     4 | 4  0 | 4  1 | 4   
A81_E303_3 AMR_NDM_x.10_NDM42_NDM46    1 | 4     4 | 4  0 | 4  4 | 4   
           AMR_NDM_x.5_NDM27_NDM29     2 | 4     3 | 4  0 | 4  0 | 4   
           AMR_NDM_x.6_NDM33_NDM35     1 | 4     4 | 4  0 | 4  4 | 4   
           AMR_NDM_x.7_NDM33_NDM36     0 | 4     3 | 4  0 | 4  3 | 4   
           AMR_NDM_x.8_NDM33_NDM37     0 | 4     4 | 4  0 | 4  4 | 4   
           AMR_NDM_x.9_NDM42_NDM45     0 | 4     2 | 4  0 | 4  3 | 4   

                                                                       \
                                    #Tm Specif       ID Template Name   
A81_E303_1 AMR_NDM_10.x_NDM31_NDM39      4 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_12.x_NDM32_NDM39      4 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_14.x_NDM40_NDM48      4 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_16.x_NDM41_NDM47      4 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_18.x_NDM41_NDM49      4 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_7.x_NDM25_NDM30       4 | 4  K.pneumoniae-NDM_2146   
A81_E303_2 AMR_NDM_x.10_NDM42_NDM46      4 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_x.5_NDM27_NDM29       4 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_x.6_NDM33_NDM35       0 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_x.7_NDM33_NDM36       4 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_x.8_NDM33_NDM37       0 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_x.9_NDM42_NDM45       4 | 4  K.pneumoniae-NDM_2146   
A81_E303_3 AMR_NDM_x.10_NDM42_NDM46      2 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_x.5_NDM27_NDM29       4 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_x.6_NDM33_NDM35       0 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_x.7_NDM33_NDM36       2 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_x.8_NDM33_NDM37       0 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_x.9_NDM42_NDM45       3 | 4  K.pneumoniae-NDM_2146   

                                                                              \
                                      Max Ct  Max Tm1  Mean Ct Mean NS ng/ul   
A81_E303_1 AMR_NDM_10.x_NDM31_NDM39  33.0946  92.8738  32.5229             0   
           AMR_NDM_12.x_NDM32_NDM39  33.4144  92.7368  32.8169      0.206305   
           AMR_NDM_14.x_NDM40_NDM48  31.9029  92.3259  30.8926             0   
           AMR_NDM_16.x_NDM41_NDM47  33.6242  92.3259  32.7265             0   
           AMR_NDM_18.x_NDM41_NDM49  32.5776  92.3259  32.2414       0.11191   
           AMR_NDM_7.x_NDM25_NDM30   31.9606  92.4629  30.9516             0   
A81_E303_2 AMR_NDM_x.10_NDM42_NDM46  31.0532  92.1705  30.0672           nan   
           AMR_NDM_x.5_NDM27_NDM29   32.8541  89.7065   29.735       1.56786   
           AMR_NDM_x.6_NDM33_NDM35   23.8407  76.7021  22.0299             0   
           AMR_NDM_x.7_NDM33_NDM36    32.549  92.3074   32.152             0   
           AMR_NDM_x.8_NDM33_NDM37   22.8593  82.3146  20.3792     0.0958177   
           AMR_NDM_x.9_NDM42_NDM45   32.9799  92.0336  31.4969             0   
A81_E303_3 AMR